In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from ema_workbench import RealParameter, ScalarOutcome, Constant, Model
from dps_lake_model import lake_model

model = Model("lakeproblem", function=lake_model)

# specify uncertainties
model.uncertainties = [
    RealParameter("b", 0.1, 0.45),
    RealParameter("q", 2.0, 4.5),
    RealParameter("mean", 0.01, 0.05),
    RealParameter("stdev", 0.001, 0.005),
    RealParameter("delta", 0.93, 0.99),
]

# set levers
model.levers = [
    RealParameter("c1", -2, 2),
    RealParameter("c2", -2, 2),
    RealParameter("r1", 0, 2),
    RealParameter("r2", 0, 2),
    RealParameter("w1", 0, 1),
]

# specify outcomes
model.outcomes = [
    ScalarOutcome("max_P"),
    ScalarOutcome("utility"),
    ScalarOutcome("inertia"),
    ScalarOutcome("reliability"),
]

# override some of the defaults of the model
model.constants = [
    Constant("alpha", 0.41),
    Constant("nsamples", 150),
    Constant("myears", 100),
]

In [4]:
from ema_workbench import MultiprocessingEvaluator, ema_logging, perform_experiments

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model, n_processes=7) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=2, policies=2)

[MainProcess/INFO] performing 2 scenarios * 2 policies * 1 model(s) = 4 experiments
  0%|                                                    | 0/4 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially
[MainProcess/ERROR] invalid index to scalar variable.
Traceback (most recent call last):
  File "C:\Users\Fabia\Anaconda3\envs\ox\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Fabia\Anaconda3\envs\ox\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Fabia\Anaconda3\envs\ox\lib\site-packages\ema_workbench\em_framework\model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "C:\Users\Fabia\Anaconda3\envs\ox\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Fabia\Anaconda3\envs\ox\lib\site-package

EMAError: exception in run_model
Caused by: IndexError: invalid index to scalar variable.

In [ ]:
experiments, outcomes = results
print(experiments.shape)
print(list(outcomes.keys()))

In [ ]:
experiments

In [ ]:
experiments.c1.unique()

In [ ]:
outcomes

In [ ]:
policies = experiments["policy"]
for i, policy in enumerate(np.unique(policies)):
    experiments.loc[policies == policy, "policy"] = str(i)

data = pd.DataFrame(outcomes)
data["policy"] = policies

In [ ]:
sns.pairplot(data, hue="policy", vars=list(outcomes.keys()))
plt.show()

In [ ]:
from SALib.analyze import sobol
from ema_workbench import Samplers
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

with MultiprocessingEvaluator(model) as evaluator:
    sa_results = evaluator.perform_experiments(
        scenarios=1000, uncertainty_sampling=Samplers.SOBOL
    )

experiments, outcomes = sa_results

problem = get_SALib_problem(model.uncertainties)
Si = sobol.analyze(
    problem, outcomes["max_P"], calc_second_order=True, print_to_console=False
)

[MainProcess/INFO] pool started with 8 workers
C:\Users\Fabia\Anaconda3\envs\ox\lib\site-packages\SALib\sample\saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000) is equal to `2^n`.
        
  warnings.warn(msg)
[MainProcess/INFO] performing 12000 scenarios * 1 policies * 1 model(s) = 12000 experiments

 27%|█████████▊                           | 3198/12000 [01:15<02:55, 50.19it/s]